# Naive implementation of Model
Idea: start with fast feedback loop in one messy notebook, afterwards organize in folder structure and adapt from there on

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.figure

## 1. Create and visualize some input

In [ ]:
def plot_bars(A, W, l=9, r=1.3, verbose=True, dpi=500, axis = None):
    """
    Plots a grid of bars with given angles, contrasts, and saliency (linewidths).
    
    Parameters:
        A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
        W (np.ndarray or None): 2D array of linewidths, same shape as A
        l (float): Bar length
        r (float): Grid spacing factor
        verbose (bool): If True, show the plot
        dpi (int): Dots per inch for rendering
        axis (matplotlib axis or None): If provided, plot on this axis instead of creating a new figure.
        
    Returns:
        fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    assert A.ndim == 2, "A must be a 2D array"
    assert W.shape == A.shape, "C must have the same shape as A"
    N_y, N_x = A.shape
    
    # Calculate image size in pixels
    d = l * r # grid spacing
    img_height = int(N_y * d)
    img_width = int(N_x * d)

    # Create figure
    if axis is not None:
        ax = axis
        fig = None
    else:
        fig, ax = plt.subplots(figsize=(img_width/100, img_height/100), dpi=dpi)
    ax.set_xlim(0, img_width)
    ax.set_ylim(0, img_height)
    ax.set_aspect('equal') # keep x and y scales the same, avoding distortion
    ax.axis('off')

    # Draw bars
    for i in range(N_y):
        for j in range(N_x):
            # compute center of the bar
            cx = (j + 0.5) * d
            cy = (i + 0.5) * d
            # compute bar directions
            angle = A[i, j]
            dx = l * np.sin(angle) / 2
            dy = l * np.cos(angle) / 2
            # compute endpoints of the bar
            x0, y0 = cx - dx, cy - dy
            x1, y1 = cx + dx, cy + dy
            # draw the bar
            ax.plot([x0, x1], [y0, y1], 
                    color = "k", 
                    linewidth=W[i, j], 
                    solid_capstyle='butt'
            )
    
    if verbose:
        plt.show()
    
    return fig

def visualize_input(A, C, l=9, r=1.3, verbose=True, dpi=500, axis=None):
    """ 
        Visualizes the input angles A and contrasts C as a grid of bars.
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
            C (np.ndarray): 2D array of contrasts, same shape as A, values in [1, 4]
            l (float): Bar length
            r (float): Grid spacing factor
            verbose (bool): If True, show the plot
            dpi (int): Dots per inch for rendering
            axis (matplotlib axis or None): If provided, plot on this axis instead of creating a new figure.
        
        Returns:
            fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    
    assert np.all((C >= 1) & (C <= 4) | (C == 0)), "C values must 0 or in [1, 4]"
    W = C / 3
    return plot_bars(A, W, l=l, r=r, verbose=verbose, dpi=dpi, axis=axis)

def visualize_output(A, S, l=9, r=1.3, verbose=True, dpi=500, axis=None):
    """ 
        Visualizes the output saliency S as a grid of bars with uniform orientation.
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
            S (np.ndarray): 3D array (Y x X) of saliency values, shape (N_y, N_x)
            l (float): Bar length
            r (float): Grid spacing factor
            verbose (bool): If True, show the plot
            dpi (int): Dots per inch for rendering
        
        Returns:
            fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    # TODO: how to scale and normalize when reading out S?
    assert np.all(S >= 0), "S values must be non-negative"
    return plot_bars(A, S, l=l, r=r, verbose=verbose, dpi=dpi, axis=axis)

In [ ]:
def bar_without_surround():
    C = np.zeros((9, 9)) 
    C[4, 4] = 3.5
    A = np.zeros((9, 9))
    return A, C

def iso_orientation():
    C = np.full((9, 9), 3.5)
    A = np.zeros((9, 9))
    return A, C

def random_background(seed=None):
    C = np.full((9, 9), 3.5)
    rng = np.random.default_rng(seed)
    A = rng.uniform(0, np.pi, (9, 9))
    A[4, 4] = 0.
    return A, C

def cross_orientation():
    C = np.full((9, 9), 3.5)
    A = np.full((9, 9),  np.pi / 2)
    A[4, 4] = 0
    return A, C

def bar_without_surround_low_contrast():
    C = np.zeros((9, 9))
    C[4, 4] = 1.05
    A = np.zeros((9, 9))
    return A, C

def with_one_flanker():
    C = np.zeros((9, 9))
    C[4, 4] = 1.05
    C[5, 4] = 3.5
    A = np.zeros((9, 9))
    return A, C

def with_two_flankers():
    C = np.zeros((9, 9))
    C[4, 4] = 1.5
    C[3, 4] = 3.5
    C[5, 4] = 3.5
    A = np.zeros((9, 9))
    return A, C

def with_flanking_line_and_noise(seed=None):
    rng = np.random.default_rng(seed)
    A = rng.uniform(0, np.pi, (9, 9))
    A[:, 4] = 0.
    C = np.full((9, 9), 3.5)
    C[4, 4] = 1.5
    return A, C

def neighboring_textures():
    A = np.zeros((11, 27))
    A[:, :14] = np.pi/2
    C = np.full((11, 27), 2.0)
    return A, C

In [ ]:
dpi = 400
N_y, N_x = 9, 9
l = 9
r = 1.3
d = l * r # grid spacing
img_height = int(N_y * d)
img_width = int(N_x * d)

plt.rcParams.update({'font.size': 6})
fig, axes = plt.subplots(2, 4, figsize=(img_width/100 * 4, img_height/100 * 2), dpi=dpi, constrained_layout=True)

axes = axes.flatten()

test_cases = [
    ("A: Bar without\nsurround", bar_without_surround),
    ("B: Iso-\norientation", iso_orientation),
    ("C: Random\nbackground", random_background),
    ("D: Cross-\norientation", cross_orientation),
    ("E: Bar without\nsurround", bar_without_surround_low_contrast),
    ("F: With one\nflanker", with_one_flanker),
    ("G: With two\nflankers", with_two_flankers),
    ("E: With flanking\nline and noise", with_flanking_line_and_noise),
]

for ax, (title, func) in zip(axes, test_cases):
    A, C = func()
    visualize_input(A, C, verbose=False, axis=ax)
    ax.set_title(title)

plt.show()

## 2. Implement Tuning Curve

In [ ]:
def tuning_curve(angle : np.ndarray) -> np.ndarray:
    """ Tuning curve function
    
    Parameters:
        angle (np.ndarray): angle difference (radians), shape (N_y, N_x, ...), values in [-pi/2, +pi/2]
    
    Returns:
        (np.ndarray): tuning curve values, shape (N_y, N_x, ...), values in [0, 1]
    """
    absolute_angle = np.abs(angle)
    absolute_angle = np.minimum(absolute_angle, np.pi - absolute_angle) # wrap to [0, pi/2]
    phi = np.exp(- absolute_angle / (np.pi / 8))
    phi[absolute_angle >= np.pi/6] = 0
    return phi

In [ ]:
k = 10000
angles_1 = np.linspace(-np.pi/2 - np.pi/k, 0, k)
angles_2 = - angles_1[::-1][1:]
angles = np.concatenate([angles_1, angles_2])
tc_values = tuning_curve(angles)
print(np.allclose(tc_values, tc_values[::-1], atol=1e-6))  # Should be True for perfect symmetry

plt.rcParams.update({'font.size': 17})
plt.figure(figsize=(6, 4), dpi = 1000, constrained_layout=True)
plt.plot(angles, tc_values, linewidth = 4)
plt.xlabel(r"Angle $x$ (radians)")
plt.ylabel(r"$\phi(x)$")
plt.title(r"Tuning Curve $\phi(x)$")
plt.grid(True)
plt.show()

In [ ]:
from typing import Optional

def get_model_input(A : np.ndarray, C : np.ndarray, M : Optional[np.ndarray] = None, K = 12) -> np.ndarray:
    """ Computes model input from visual input

    TODO: extend to multiple input bars per locationn (i.e. A and C of shape (N_y, N_x, L) where L is number of input bars per location)
    
    Parameters:
        A (np.ndarray): 2D array of angles (radians) of input bars, shape (N_y, N_x), values in [0, pi]  
        C (np.ndarray): 2D array of contrasts of input bars, same shape as A, values in [1, 4] or 0 (no bar)
        M (np.ndarray): prefered orientations of model neurons, shape (N_y, N_x, K), values in [0, pi], where K is number of orientation channels
    
    Returns:
        I (np.ndarray): 3D array of model input, shape (K, N_y, N_x)
    
    """
    if M is None:
        angles = np.linspace(0, np.pi, K, endpoint=False) 
        M = angles[np.newaxis, np.newaxis, :]
        N_y, N_x = A.shape
        M = np.broadcast_to(M, (N_y, N_x, K))
    
    M = M % np.pi  # ensure M in [0, pi]
    A = A % np.pi  # ensure A in [0, pi]
    
    A = A[:, :, np.newaxis]  # shape (N_y, N_x, 1)
    C = C[:, :, np.newaxis]  # shape (N_y, N_x, 1)
    return C * tuning_curve(A - M), M

In [ ]:
test = False

# Generate random valid inputs
N_y, N_x = 3, 9
rng = np.random.default_rng(42)
A_test = rng.uniform(0, np.pi, (N_y, N_x))
C_test = np.full((N_y, N_x), 2.5) # rng.uniform(1, 4, (N_y, N_x))

# Call get_model_input
I, M = get_model_input(A_test, C_test)

print("Input shape:", I.shape)
print("Input min/max:", I.min(), I.max())

for k in range(I.shape[2]):
    print(M[0, 0, k] / np.pi * 180)
    visualize_output(M[:, :, k], I[:, :, k], verbose=test)

## 3. Implement Naive Model

In [ ]:
class NaiveModel:
    def __init__(self, K=12, alpha=1.0):
        self.alpha = alpha
        self.K = K
        
        # Precompute preferred orientations per neuron
        angles = np.linspace(0, np.pi, self.K, endpoint=False) 
        self.M = angles[np.newaxis, np.newaxis, :] # shape (1, 1, K)
    
    def get_input(self, A: np.ndarray, C: np.ndarray, verbose : bool = False) -> np.ndarray:
        """ Computes model input from visual input

        TODO: extend to multiple input bars per locationn (i.e. A and C of shape (N_y, N_x, L) where L is number of input bars per location)
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians) of input bars, shape (N_y, N_x), values in [0, pi]  
            C (np.ndarray): 2D array of contrasts of input bars, same shape as A, values in [1, 4] or 0 (no bar)
            verbose (bool): If True, visualize input and output
            
        Returns:
            I (np.ndarray): 3D array of model input, shape (N_y, N_x, K)
        
        """
        M = np.broadcast_to(self.M, (N_y, N_x, self.K))
        M = M % np.pi  # ensure M in [0, pi]
        
        A = A % np.pi  # ensure A in [0, pi]
        A = A[:, :, np.newaxis]  # shape (N_y, N_x, 1)
        C = C[:, :, np.newaxis]  # shape (N_y, N_x, 1)
        
        I = C * tuning_curve(A - M)
        
        if verbose:
            visualize_input(A, C, verbose=True)
            for k in range(I.shape[2]):
                print("==================================")
                print(f"Neurons {k}, tuned to {M[0, 0, k] / np.pi * 180}°")
                visualize_output(M[:, :, k], I[:, :, k], verbose=True)
        
        return I
    
    def derivative(self, X: np.ndarray, I: np.ndarray) -> np.ndarray:
        """ Computes the derivative dX/dt 
        
        Parameters:
            X (np.ndarray): Current state, shape (N_y, N_x, K)
            I (np.ndarray): Input, shape (N_y, N_x, K)
        
        Returns:
            (np.ndarray): Derivative dX/dt, shape (N_y, N_x, K)
        """
        
        return - self.alpha * X + I
        
    def euler_method(self, I: np.ndarray, dt: float, T: float) -> np.ndarray:
        """ Simulates the model over time given input I
        
        Parameters:
            I (np.ndarray): Input, shape (N_y, N_x, K)
            dt (float): Time step
            T (float): Total simulation time
        
        Returns:
            X (np.ndarray): Final state after simulation, shape (T, N_y, N_x, K)
        """
        
        steps = int(T // dt)
        X = np.zeros((steps, *I.shape))
        
        X[0] = np.zeros_like(I)  # initial state = input
        for t in range(1, steps):
            dXdt = self.derivative(X[t-1], I)
            X[t] = X[t-1] + dt * dXdt
        
        return X
    
    def simulate(self, A: np.ndarray, C: np.ndarray, dt: float = 0.001, T: float = 12.0, verbose: bool = False) -> np.ndarray:
        """ Runs the full simulation given angles A and contrasts C
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians) of input bars, shape (N_y, N_x), values in [0, pi]  
            C (np.ndarray): 2D array of contrasts of input bars, same shape as A, values in [1, 4] or 0 (no bar)
            dt (float): Time step
            T (float): Total simulation time
        
        Returns:
            X (np.ndarray): Final state after simulation, shape (T, N_y, N_x, K)
        """
        
        I = self.get_input(A, C, verbose=verbose)
        X = self.euler_method(I, dt, T)
        return X, I

In [ ]:
# Instantiate and test the NaiveModel
model = NaiveModel(K=10, alpha=1.0)

A = np.array([[90 / 180 * np.pi]])
C = np.array([[2.0]])

T = 8
dt = 0.001
X_gen, I = model.simulate(A, C, dt=dt, T=T, verbose=False)

def analytical_solution(t: np.ndarray, I: np.ndarray, alpha: float) -> np.ndarray:
    """ Computes the analytical solution of the ODE at time t given input I and parameter alpha.
        Assumes constant input, and initial condition X(t = 0) = 0.
    
    Parameters:
        t (np.ndarray): Time points, shape (N_t,)
        I (np.ndarray): Input, shape (N_y, N_x, K)
        alpha (float): Model parameter
    
    Returns:
        (np.ndarray): Analytical solution at time t, shape (N_y, N_x, K)
    """
    t_ = t[:, np.newaxis, np.newaxis, np.newaxis]  # shape (N_t, 1, 1, 1)
    I_ = I[np.newaxis, :, :, :]  # shape (1, N_y, N_x, K)
    return (1 - np.exp(-alpha * t_)) / alpha * I_

t = np.arange(X_gen.shape[0]) * dt
X_gt = analytical_solution(t, I, model.alpha)

plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(6, 4), dpi=300, constrained_layout=True)
labels = [f"{round(180 / model.K * k)}°" for k in range(model.K)]
colors = plt.cm.hsv(np.linspace(0, 1, model.K, endpoint=False))  # Use twilight colormap
linestyles = ['-', '--', '-.', ':'] * ((model.K // 4) + 1)  # Cycle through styles
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h'] * ((model.K // 10) + 1)  # Cycle through markers

for k in range(model.K):
    plt.plot(
        t, X_gen[:, 0, 0, k],
        linestyle=linestyles[k],
        label=f"{labels[k]}",
        color=colors[k],
        alpha=0.8,
        linewidth=2,
        marker=markers[k],
        markevery=(k * round(len(t)/6/model.K), round(len(t)/6)),  # Offset markers for each line
        markeredgecolor='black',      # Add black outline
        markeredgewidth=0.6 
    )
    plt.plot(
        t, X_gt[:, 0, 0, k],
        linestyle='-',
        color="k",
        alpha=0.1,
        linewidth=4,
        label = "Analytical\nSolutions" if k == model.K - 1 else None
    )
plt.legend(loc='center right', title="Preferred\norientation", framealpha=1.0, fontsize=8, title_fontsize=9)
plt.xlabel("Time [per time constant]")
plt.ylabel("Model response")
plt.title("Response of model neurons in one hypercolumn\n" + r"to a bar of orientation $\theta = 90^\circ$ and contrast $\hat{I} = 2$")
plt.show()

In [ ]:
# Time points in seconds to plot
time_points = [0.7, 0.9, 1.2, 1.4, 1.8, 6]
steps = [int(t / dt) for t in time_points]

# average of input across columns and for the orientation of the target bar
X_per_column = np.concatenate([X_gen[:, :, :14, 6].mean(axis = 1), X_gen[:, :, 14:, 0].mean(axis = 1)], axis=1)

visualize_input(A, C, verbose=True)

# Plot
plt.figure(figsize=(10, 6))
x_axis = np.arange(X_per_column.shape[1])  # column indices

for t_idx, step in zip(time_points, steps):
    # Neural response: sum over orientation channels (axis=-1)
    response = X_per_column[step]
    plt.plot(x_axis, response, label=f't={t_idx:.1f}s')

# Plot temporal average
avg_response = X_per_column.mean(axis=0)
plt.plot(x_axis, avg_response, label='Temporal avg', linewidth=3, linestyle='--', color='k')

plt.xlabel('Texture column number')
plt.ylabel('Neural response')
plt.legend()
plt.grid(True)
plt.show()

# 4. Implement noise model

In [ ]:
class NaiveNoisyModel:
    def __init__(self, K=12, alpha=1.0, average_noise_height=0.1, average_noise_temporal_width=0.1, seed=None):
        self.alpha = alpha
        self.K = K
        
        # Precompute preferred orientations per neuron
        angles = np.linspace(0, np.pi, self.K, endpoint=False) 
        self.M = angles[np.newaxis, np.newaxis, :] # shape (1, 1, K)
        
        # Noise parameters
        assert average_noise_height >= 0, "average_noise_height must be non-negative"
        assert average_noise_temporal_width > 0, "average_noise_temporal_width must be positive"
        self.noise_std = average_noise_height
        self.noise_tau = average_noise_temporal_width
        self.rng = np.random.default_rng(seed)
    
    def get_input(self, A: np.ndarray, C: np.ndarray, verbose : bool = False) -> np.ndarray:
        """ Computes model input from visual input

        TODO: extend to multiple input bars per locationn (i.e. A and C of shape (N_y, N_x, L) where L is number of input bars per location)
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians) of input bars, shape (N_y, N_x), values in [0, pi]  
            C (np.ndarray): 2D array of contrasts of input bars, same shape as A, values in [1, 4] or 0 (no bar)
            verbose (bool): If True, visualize input and output
            
        Returns:
            I (np.ndarray): 3D array of model input, shape (N_y, N_x, K)
        
        """
        assert A.ndim == 2, "A must be a 2D array"
        assert C.shape == A.shape, "C must have the same shape as A"
        N_y, N_x = A.shape
        M = np.broadcast_to(self.M, (N_y, N_x, self.K))
        M = M % np.pi  # ensure M in [0, pi]
        
        A = A % np.pi  # ensure A in [0, pi]
        A = A[:, :, np.newaxis]  # shape (N_y, N_x, 1)
        C = C[:, :, np.newaxis]  # shape (N_y, N_x, 1)
        
        I = C * tuning_curve(A - M)
        
        if verbose:
            visualize_input(A, C, verbose=True)
            for k in range(I.shape[2]):
                print("==================================")
                print(f"Neurons {k}, tuned to {M[0, 0, k] / np.pi * 180}°")
                visualize_output(M[:, :, k], I[:, :, k], verbose=True)
        
        return I
        
    
    def derivative(self, X: np.ndarray, I: np.ndarray) -> np.ndarray:
        """ Computes the derivative dX/dt 
        
        Parameters:
            X (np.ndarray): Current state, shape (N_y, N_x, K)
            I (np.ndarray): Input, shape (N_y, N_x, K)
        
        Returns:
            (np.ndarray): Derivative dX/dt, shape (N_y, N_x, K)
        """
        
        return - self.alpha * X + I
        
    def euler_method(self, I: np.ndarray, dt: float, T: float) -> np.ndarray:
        """ Simulates the model over time given input I
        
        Parameters:
            I (np.ndarray): Input, shape (N_y, N_x, K)
            dt (float): Time step
            T (float): Total simulation time
        
        Returns:
            X (np.ndarray): Final state after simulation, shape (T, N_y, N_x, K)
        """
        
        steps = int(T // dt)
        X = np.zeros((steps, *I.shape))
        X[0] = np.zeros_like(I)  # initial state = input
        
        I_noise = np.zeros_like(I)
        noise_duration = np.zeros_like(I)
        
        for t in range(1, steps):
            # dXdt = self.derivative(X[t-1], I)
            # X[t] = X[t-1] + dt * dXdt
            
            # add noise: temporal width follows exponential distribution, amplitude follows normal distribution
            noise_duration -= dt
            I_noise[noise_duration <= 0] = self.rng.normal(0, self.noise_std, size=I.shape)[noise_duration <= 0]
            noise_duration[noise_duration <= 0] = self.rng.exponential(self.noise_tau, size=I.shape)[noise_duration <= 0]
            X[t] += I_noise * dt
        
        return X
    
    def simulate(self, A: np.ndarray, C: np.ndarray, dt: float = 0.001, T: float = 12.0, verbose: bool = False) -> np.ndarray:
        """ Runs the full simulation given angles A and contrasts C
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians) of input bars, shape (N_y, N_x), values in [0, pi]  
            C (np.ndarray): 2D array of contrasts of input bars, same shape as A, values in [1, 4] or 0 (no bar)
            dt (float): Time step
            T (float): Total simulation time
        
        Returns:
            X (np.ndarray): Final state after simulation, shape (T, N_y, N_x, K)
        """
        
        I = self.get_input(A, C, verbose=verbose)
        X = self.euler_method(I, dt, T)
        return X, I

In [ ]:
# Instantiate and test the NaiveModel
model = NaiveNoisyModel(K=12, alpha=1.0, average_noise_height=0.1, average_noise_temporal_width=0.1)

A = np.full((9, 9), 90 / 180 * np.pi)
C = np.zeros((9, 9))

T = 10
dt = 0.001
X_gen, I = model.simulate(A, C, dt=dt, T=T, verbose=False)

discard_initial_steps = int(model.noise_tau / dt * 10)
X_gen = X_gen[discard_initial_steps:]

In [ ]:
np.allclose(model.noise_std, np.sqrt(np.mean(X_gen**2, axis=0)).mean() / dt, atol = 1e-3)

In [ ]:
from scipy.signal import correlate

# Compute autocorrelation for lags up to 100
n_lags = 1000
X_gen_centered = X_gen - np.mean(X_gen, axis=0)
autocorrs = np.zeros((n_lags + 1, X_gen.shape[1], X_gen.shape[2], X_gen.shape[3]))
for m_x in range(X_gen.shape[2]):
    for m_y in range(X_gen.shape[1]):
        for k in range(X_gen.shape[3]):
            autocorr = correlate(X_gen_centered[:, m_y, m_x, k], X_gen_centered[:, m_y, m_x, k], method='auto')
            autocorr = autocorr[autocorr.size // 2:autocorr.size // 2 + n_lags + 1]
            # autocorr /= (X_gen.shape[0] - np.arange(n_lags + 1))
            autocorr /= autocorr[0]  # Normalize
            autocorrs[:, m_y, m_x, k] = autocorr

In [ ]:
lags = np.arange(n_lags + 1) * dt

def analytical_autocorrelation(lags: np.ndarray, tau: float) -> np.ndarray:
    """ Computes the analytical autocorrelation function for the noise process.
    
    Parameters:
        lags (np.ndarray): Lag times, shape (N_lags,)
        tau (float): Temporal width of the noise process
    
    Returns:
        (np.ndarray): Analytical autocorrelation values, shape (N_lags,)
    """ 
    return np.exp(- lags / tau) # * (X_gen.shape[0] - lags) / (X_gen.shape[0])

plt.figure(figsize=(8, 4))
mean = autocorrs.mean(axis=(1,2,3))
plt.plot(lags, mean, color='blue', linewidth=2, label="Simulation")
sem = autocorrs.std(axis=(1,2,3)) / np.sqrt(autocorrs.shape[1] + autocorrs.shape[2] + autocorrs.shape[3])
plt.fill_between(lags, 
                 mean - sem, 
                 mean + sem, 
                 color='blue', alpha=0.2)
plt.plot(lags, analytical_autocorrelation(lags, model.noise_tau), color='red', linewidth=2, label='Analytical', linestyle = "--")
plt.title("Autocorrelation of $X_{gen}$")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.legend(loc='upper right', framealpha=1.0)
plt.grid(True)
plt.xlim(0, 0.1)
plt.show()

TODO: debug this

In [ ]:
t = np.arange(X_gen.shape[0]) * dt
X_gt = analytical_solution(t, I, model.alpha)

plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(6, 4), dpi=300, constrained_layout=True)
labels = [f"{round(180 / model.K * k)}°" for k in range(model.K)]
colors = plt.cm.hsv(np.linspace(0, 1, model.K, endpoint=False))  # Use twilight colormap
linestyles = ['-', '--', '-.', ':'] * ((model.K // 4) + 1)  # Cycle through styles
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h'] * ((model.K // 10) + 1)  # Cycle through markers

for k in range(model.K):
    plt.plot(
        t, X_gen[:, 0, 0, k],
        linestyle="-",
        label=f"{labels[k]}",
        color=colors[k],
        alpha=0.8,
        linewidth=2,
        marker=markers[k],
        markevery=(k * round(len(t)/6/model.K), round(len(t)/6)),  # Offset markers for each line
        markeredgecolor='black',      # Add black outline
        markeredgewidth=0.6 
    )
    plt.plot(
        t, X_gt[:, 0, 0, k],
        linestyle='-',
        color="k",
        alpha=0.1,
        linewidth=4,
        label = "Analytical\nSolutions" if k == model.K - 1 else None
    )
plt.legend(loc='center right', title="Preferred\norientation", framealpha=1.0, fontsize=8, title_fontsize=9)
plt.xlabel("Time [per time constant]")
plt.ylabel("Model response")
plt.title("Response of model neurons in one hypercolumn\n" + r"to a bar of orientation $\theta = 90^\circ$ and contrast $\hat{I} = 2$")
plt.show()